# Novelty Generation Writer Transcripts

Consume existing media, manipulate to produce new novel media.


In [9]:
import copy 
import os
import subprocess
import json
import sys
import numpy as np
import cv2
import glob

JSON_FILE_LOCATIONS = "../data/algorithms"
DATA_LOCATION = '/data/iam_data/letter_clean'
RESULT = '/data/iam_data/'

In [11]:
# Load up the background images to use into memory
background_img_loc=['../data/images/antique_bg.jpg']

background_img_val = []
for item in background_img_loc:
    background_img_val.append(cv2.imread(item))

In [12]:


def letter_image_op(text_image, bg_image_id=0):
    """
    change the letter (pen)
    """
    #set offsets to random part of background
    bg_image = background_img_val[bg_image_id]
  
    if bg_image.shape[1] >= text_image.shape[1]:
        x_offset = int(random.random() * (bg_image.shape[1] - text_image.shape[1]))
    else:
        x_offset = 0 
        
    if bg_image.shape[0] >= text_image.shape[0]:
        y_offset = int(random.random() * (bg_image.shape[0] - text_image.shape[0]))
    else:
        y_offset = 0
        
    bg_image_select = bg_image[y_offset:y_offset+text_image.shape[0],
                               x_offset:x_offset+text_image.shape[1],:]

    if text_image.shape[0:2] != bg_image.shape[0:2]:
        bg_image_select= cv2.resize(bg_image_select,(text_image.shape[1],text_image.shape[0]))
        
    blended = np.copy(text_image)
    img_max = np.max(text_image,axis=2)
    for i in range(text_image.shape[2]):
        blended[img_max<250,i] = bg_image_select[img_max<250,i]
    return blended

def bg_image_op(text_image, bg_image_id=0, approach='minimum', color=[255,255,255]):
    """
    change the background (paper)
    """
    #set offsets to random part of background
    bg_image = background_img_val[bg_image_id]
  
    if bg_image.shape[1] >= text_image.shape[1]:
        x_offset = int(random.random() * (bg_image.shape[1] - text_image.shape[1]))
    else:
        x_offset = 0 
        
    if bg_image.shape[0] >= text_image.shape[0]:
        y_offset = int(random.random() * (bg_image.shape[0] - text_image.shape[0]))
    else:
        y_offset = 0
        
    bg_image_select = bg_image[y_offset:y_offset+text_image.shape[0],
                               x_offset:x_offset+text_image.shape[1],:]

    if text_image.shape[0:2] != bg_image.shape[0:2]:
        bg_image_select= cv2.resize(bg_image_select,(text_image.shape[1],text_image.shape[0]))
        
    blended = np.copy(text_image)
    for i in range(text_image.shape[2]):
        if approach == "minimum":
            blended[:,:,i] = np.minimum(text_image[:,:,i],bg_image_select[:,:,i])
        elif approach == "factor":
            img = np.copy(text_image[:,:,i])
            # darker is letters
            factor = (255-img)/255.0
            blended[:,:,i] = bg_image_select[:,:,i]*(1-factor) + img*factor
        else:
            img = np.copy(text_image[:,:,i])
            blended[:,:,i] = bg_image_select[:,:,i]
            blended[:,:,i][img<250] = color[i]*((255-img)/255.0).astype(np.uint8)[img<250]
    return blended

def lines_color(image, color=[255,255,255]):
    img_max = np.max(image,axis=2)
    adjusted = np.copy(image)
    adjusted[img_max<250,:] = color
    return adjusted

def bg_color(image, color=[255,255,255]):
    img_min = np.max(image,axis=2)
    adjusted = np.copy(image)
    adjusted[img_min>250,:] = color
    return adjusted

def invert_image(image):
    return 255-image

def gaussian_noise(image, var=0.5):
    row,col,ch= image.shape
    mean = 0
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))*10
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return noisy.astype(np.uint8)

def flip(image):
    return cv2.flip(image)


def scale(image, transform_matrix = np.asarray([[1.5,1.5,0],[0,1,0,],[0,0,1]]).astype(float)):
    return cv2.warpPerspective(image, 
                                    transform_matrix, 
                                    (image.shape[1], image.shape[0]), 
                                    flags=cv2.INTER_LINEAR,
                                    borderMode=cv2.BORDER_CONSTANT, 
                                    borderValue=[255,255,255])



In [13]:


def _run_all(source_dir, dest, op, op_args):
    images = glob.glob(os.path.join(source_dir, "**", "*.png"), recursive=True)
    for input_img in images:
        output_img_fn = os.path.join(dest, input_img[len(source_dir)+1:])
        if os.path.exists(output_img_fn):
            continue
        os.makedirs(os.path.dirname(output_img_fn), exist_ok=True)
        print(output_img_fn)
        out_img = op(cv2.imread(input_img),**op_args)
        if out_img is not None:
            cv2.imwrite(output_img_fn, out_img) 
    
def run_choice(dest_dir, source_dir, choice, op, op_args):
    dest = dest_dir + choice
    if not os.path.exists(dest):
        os.mkdir(dest)
    _run_all(source_dir, dest, op, op_args)
    

In [14]:
dir_pairs = {
             'bg_blue_wall_red_letter': (bg_image_op, {'bg_image_id':0, 'approach': 'overwrite', 'color': (30,40,200) }), 
             'bg_antique': (bg_image_op, {'bg_image_id':0}), 
             'bg_gaussian_noise': (gaussian_noise, {'var': 0.5}),
             'letter_inverted': (invert_image,{}),
             'bg_blue_color': (bg_color,{'color': (200,30,40)}),
             'letter_blue': (lines_color,{'color': (200,30,40)}),
             'letter_red': (lines_color,{'color': (30,40,200)}),
             'letter_flip': (flip, {}),
             'letter_scale': (scale, { 'transform_matrix': np.asarray([[1.5,1.5,0],[0,1,0,],[0,0,1]]).astype(float)}),
             'letter_slant_small': (scale, { 'transform_matrix': np.asarray([[0.5,0.5,0],[0,0.75,0,],[0,0,1]]).astype(float)}),
            }

In [ ]:
choices = dir_pairs.keys()

if len(choices) > 0:
    for choice in choices:
        run_choice(ROOT,IMAGE_DIR, choice, dir_pairs[choice][0], dir_pairs[choice][1])
else:
    for k in dir_pairs:
        print(k)
        dest = ROOT + k
        if os.path.exists(dest):
            continue
        run_choice(ROOT,IMAGE_DIR, k, dir_pairs[k][0], dir_pairs[k][1])